In [11]:
import os
from pathlib import Path

import cv2
import openvino
import pandas as pd
from loguru import logger
from tqdm import tqdm
from ultralytics import YOLO

In [12]:
DATA_DIR = "/home/ashish/solution"  # replace with the test dataset directory
OUTPUT_PATH = "/home/ashish/solution/submission.csv"  # replace where submission file needs to be saved

In [6]:
print(openvino.__version__)


def centered_box(img, scale=0.1):
    """
    Return coordinates for a centered bounding box on the image, defaulting to 10% of the image's height and width.
    """
    # Get image dimensions
    height, width, _ = img.shape
    # Calculate the center of the image
    center_x, center_y = width // 2, height // 2
    # Calculate 10% of the image's height and width for the bounding box
    box_width, box_height = width * scale, height * scale
    # Calculate top-left corner of the bounding box
    x1 = center_x - box_width // 2
    y1 = center_y - box_height // 2
    # Calculate bottom-right corner of the bounding box
    x2 = center_x + box_width // 2
    y2 = center_y + box_height // 2

    return [x1, y1, x2, y2]


def main(data_dir, output_path):
    # locate key files and locations
    data_dir = Path(data_dir).resolve()
    output_path = Path(output_path).resolve()
    submission_format_path = data_dir / "submission_format.csv"
    images_dir = data_dir / "images"

    assert data_dir.exists(), f"Data directory does not exist: {data_dir}"
    assert output_path.parent.exists(), f"Expected output directory {output_path.parent} does not exist"
    assert submission_format_path.exists(), f"Expected submission format file {submission_format_path} does not exist"
    assert images_dir.exists(), f"Expected images dir {images_dir} does not exist"
    logger.info(f"using data dir: {data_dir}")

    # copy the submission format file; we'll use this as template and overwrite placeholders with our own predictions
    submission_format_df = pd.read_csv(submission_format_path, index_col="image_id")
    submission_df = submission_format_df.copy()
    # load pretrained model we included in our submission.zip
    model = YOLO("assets/model.pt")
    model.export(format="openvino", imgsz=1280)
    model = YOLO("assets/model_openvino_model")
    # add a progress bar using tqdm without spamming the log
    update_iters = min(100, int(submission_format_df.shape[0] / 10))
    with open(os.devnull, "w") as devnull:
        progress_bar = tqdm(
            enumerate(submission_format_df.index.values),
            total=submission_format_df.shape[0],
            miniters=update_iters,
            file=devnull,
        )
        # generate predictions for each image
        for i, image_id in progress_bar:
            if (i % update_iters) == 0:
                logger.info(str(progress_bar))
            # load the image
            img = cv2.imread(str(images_dir / f"{image_id}.png"))
            # get yolo result
            result = model(img, imgsz=1280, verbose=False, conf=0.0001)[0]
            # get bbox coordinates if they exist, otherwise just get a generic box in center of an image
            bbox = result.boxes.xyxy[0].tolist() if len(result.boxes) > 0 else centered_box(img)
            # convert bbox values to integers
            bbox = [int(x) for x in bbox]
            # store the result
            submission_df.loc[image_id] = bbox

    # write the submission to the submission output path
    submission_df.to_csv(output_path, index=True)


if __name__ == "__main__":
    main(DATA_DIR, OUTPUT_PATH)

2024-05-20 19:11:13.540 | INFO     | __main__:main:47 - using data dir: /home/ashish/drivendata/solution/fold0


2024.1.0-15008-f4afc983258-releases/2024/1
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.1.2 CPU (AMD Ryzen Threadripper 3960X 24-Core Processor)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from 'assets/model.pt' with input shape (1, 3, 1280, 1280) BCHW and output shape(s) (1, 5, 33600) (64.1 MB)

OpenVINO: starting export with openvino 2024.1.0-15008-f4afc983258-releases/2024/1...
OpenVINO: export success ✅ 2.0s, saved as 'assets/model_openvino_model/' (43.0 MB)

Export complete (3.9s)
Results saved to /home/ashish/drivendata/solution/assets
Predict:         yolo predict task=detect model=assets/model_openvino_model imgsz=1280  
Validate:        yolo val task=detect model=assets/model_openvino_model imgsz=1280 data=/home/ashish/drivendata/yolov8x640/fold0/gbr.yaml  
Visualize:       https://netron.app
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=

2024-05-20 19:11:17.570 | INFO     | __main__:main:68 -   0%|          | 0/1089 [00:00<?, ?it/s]


Loading assets/model_openvino_model for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference...


2024-05-20 19:11:30.786 | INFO     | __main__:main:68 -   9%|▉         | 100/1089 [00:13<02:10,  7.57it/s]
2024-05-20 19:11:41.389 | INFO     | __main__:main:68 -  18%|█▊        | 200/1089 [00:23<01:43,  8.56it/s]
2024-05-20 19:11:51.845 | INFO     | __main__:main:68 -  28%|██▊       | 300/1089 [00:34<01:27,  8.99it/s]
2024-05-20 19:12:02.619 | INFO     | __main__:main:68 -  37%|███▋      | 400/1089 [00:45<01:15,  9.10it/s]
2024-05-20 19:12:14.017 | INFO     | __main__:main:68 -  46%|████▌     | 500/1089 [00:56<01:05,  8.98it/s]
2024-05-20 19:12:25.204 | INFO     | __main__:main:68 -  55%|█████▌    | 600/1089 [01:07<00:54,  8.97it/s]
2024-05-20 19:12:36.354 | INFO     | __main__:main:68 -  64%|██████▍   | 700/1089 [01:18<00:43,  8.97it/s]
2024-05-20 19:12:47.378 | INFO     | __main__:main:68 -  73%|███████▎  | 800/1089 [01:29<00:32,  9.00it/s]
2024-05-20 19:12:58.326 | INFO     | __main__:main:68 -  83%|████████▎ | 900/1089 [01:40<00:20,  9.05it/s]
2024-05-20 19:13:09.733 | INFO     | 